In [1]:
import dotenv
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_openai import AzureOpenAIEmbeddings
dotenv.load_dotenv()
from langchain_community.vectorstores import Chroma
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness,
)
from datasets import Dataset
from langchain_openai import AzureChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
import os
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
import getpass
from langchain.retrievers.document_compressors import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever
import sys
sys.tracebacklimit = 0
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import PyPDFLoader

c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
OPENAI_MODEL = os.environ.get("OPENAI_MODEL")
OPENAI_DEPLOYMENT = os.environ.get("OPENAI_DEPLOYMENT")
EMBEDDING_MODEL = os.environ.get("EMBEDDING_MODEL")
EMBEDDING_DEPLOYMENT = os.environ.get("EMBEDDING_DEPLOYMENT")
OPENAI_MODEL_GPT4 = os.environ.get("OPENAI_MODEL_GPT4")
OPENAI_DEPLOYMENT_GPT4 = os.environ.get("OPENAI_DEPLOYMENT_GPT4")

In [3]:
embeddings_client = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDING_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION)
llm = AzureChatOpenAI(model_name=OPENAI_MODEL, azure_deployment=OPENAI_DEPLOYMENT,temperature=0)
llm_gpt4 = AzureChatOpenAI(model_name=OPENAI_MODEL_GPT4, azure_deployment=OPENAI_DEPLOYMENT_GPT4,temperature=0)

In [4]:
def evaluation_llm(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
        raise_exceptions=False
    )
    return result

In [5]:
def evaluation_rag(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  # most likely
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
        raise_exceptions=False,
    )
    return result

In [6]:
questions = [
    "Analyze the following quote from Dr. Volumnia Gaul from 'Ballad of Songbirds and Snakes': 'There is a point to everything or nothing at all, depending on your worldview.'",
    "Analyze the following quote from 'Ballad of Songbirds and Snakes': 'He'd loved the unfamiliar sense of safety that their defeat had brought. The security that could only come with power. The ability to control things. Yes, that was what he'd loved best of all.'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'Snow lands on top.'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': '...at eighteen, the heir to the once-great house of Snow had nothing to live on but his wits.'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'Bombs and blood. That was how the rebels had killed his mother. He wondered if they had killed Lucy Gray's, too. 'Just like her pearly white bones.' She seemed to have no love for District 12, always separating herself from it, saying she was, what was it . . . Covey?'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'All right, so he'd dropped the handkerchief with Lucy Gray's scent - the one from the outside pocket of his book bag - into the snake tank. He'd done it so they would not bite her as they had Clemensia. So they would not kill her. Because he cared about her. Because he cared about her? Or because he wanted to win the Hunger Games so that he could secure the Plinth Prize. If it was the latter, he had cheated to win, and that was that.'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'The Hunger Games are an unnatural, vicious punishment. How could a good person like you be expected to go along with them?'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'If I'm helping to kill people in the districts, how is it any better than helping to kill them in the Hunger Games?'",
    "Analyze this quote from 'Ballad of Songbirds and Snakes': 'You don't think I've invested all this time in you to hand you off to those imbeciles in the districts, do you?'"
]


ground_truths = [
    ["Throughout The Ballad, Dr. Gaul presents Coryo and the other students with items of political philosophy, often in order to explain her own views of power and control. Here, her contrast between a 'point' and 'nothing' plays into a few of her larger ideas. Authoritarian though she is, Dr. Gaul sees the power of the Capitol as serving a useful point in rescuing Panem from chaos. On another level, the situation that provokes this comment - Coryo and Clemensia observing Dr. Gaul's multicolored snakes and wondering if there is a 'point' to the coloration - reveals that the idea of purpose extends well beyond vast political constructs in the novel. Dr. Gaul's rainbow snakes serve a direct point in making the Hunger Games a stark spectacle, and they serve a narrative point in connecting Lucy with one of her symbolic animals in the arena. It is clear what worldview Dr. Gaul would endorse."],
    ["Here, Coryo is completing an assignment for Dr. Gaul on the rather twisted topic of what he liked about the war, and he is proving just how compatible Dr. Gaul's authoritarian worldview is with his own. Of course, Coryo's reasons for disliking the Districts are not purely sociopathic. The warfare caused his family real suffering and gave him the impression that, despite his apparent intelligence and his potential for decency, he was, for a time, fundamentally helpless. Still, is it reassuring 'security' or maniacal 'power' that really appeals to Coriolanus? On the basis of the later events of his life, his desire for control and power becomes twisted beyond understandable self-preservation and into open villainy."],
    ["This is something of a Snow family motto for Tigris and Coryo: the 'saying that had gotten them through the war' (Ballad, 9), as the two cousins recollect early in the novel. Though loaded with reassurance and a pleasing sense of inevitability, this mantra varies in tone, context, and implication as The Ballad of Songbirds and Snakes progresses. 'Snow lands on top' appears early on, when Tigris is contemplating her family's poverty and Snow is eager to excel as a mentor. These are inspiring words for difficult circumstances, but by the final time the motto appears—in the Epilogue, with Coryo victorious and increasingly vicious—a Snow has in fact landed on top and has succeeded in crushing plenty of other people."],
    ["Here, we are told that Snow is only eighteen, and we are led to contemplate the differences between the wealthy dictator of the later Hunger Games trilogy and the earnest young Coryo we see here. His kindly mother and his domineering father are dead, and he doesn't have their fortune—or any fortune—to benefit him, since his family's resources were lost in the rebellion with the presumed destruction of District 13. Now, Snow must use his wits and cunning to escape his situation. Thus, the book sets up Coryo's need to restore his family's name as one of the character's central motivations. Readers of the final trilogy may know the end result of Snow's life, but the task of connecting this impoverished young man to the well-heeled, sociopathic President Snow offers plenty of intrigue."],
    ["Is the bond between Snow and Lucy mainly based on self-interest or on genuine affection? While a romance between an aristocrat from the Capitol and a wandering musician may seem to have a shaky foundation, Snow's early reflections indicate that he feels some sympathy for Lucy. Their shared experience of the war is one of loneliness and displacement. Still, as Snow's need to excel takes precedence, his connection to Lucy becomes more complicated. The idea that Lucy has 'no love for District 12,' for instance, will later be deployed by Snow in discussions and interviews meant to appeal to Capitol sponsors. Of course, construing Lucy as non-District may help her popularity and her odds of winning, but it is impossible to separate Snow's bond with Lucy from his own determination to improve his life."],
    ["The narration of The Ballad is by no means indirect about the moral dilemmas that Coryo encounters. Here, the narrative explicitly raises the question of whether he desires Lucy's safety or a personal win more, underscoring this theme with sentences that are identical but for a single unit of punctuation: 'Because he cared about her. Because he cared about her?' At this point in the novel, it is possible to read a single action as doubled-sided and Coryo's confusion as genuine. By the end, however, he will decisively choose his future over Lucy, reaching a level of self-interest that he doesn't definitively attain in this scene."],
    ["There are a few Capitol citizens who object to the Hunger Games in a principled manner, though not all of them speak out as sharply as Sejanus Plinth does. Among the mentors, Lysistrata Vickers seems to harbor real compassion for her tribute, and Coriolanus himself is troubled - at least for a time - by the fact that he kills a tribute while escaping the arena. Tigris's own objections are, from one perspective, surprising. Though related to a Capitol loyalist (her grandmother) and a future authoritarian (her cousin), she finds the games miserable and believes that Coryo does the same. Her judgment of Coriolanus may be flawed and her idea may not quite fit her family's sympathies."],
    ["In part, Sejanus's decision to become a Peacekeeper was meant to remove him from a Capitol milieu that he never found appealing. Nonetheless, this quotation reveals that Sejanus is in an impossible position. If becoming a Peacekeeper - ideally, a Peacekeeper doctor or medic - was meant as a form of escape and a route to atonement, then Sejanus's new approach has led him down the wrong path. He is forced to help an institution that executes rebels without trial and that, for the most part, is willing to let the Districts remain in disrepair - beyond providing the Capitol with resources, that is."],
    ["Here, Dr. Gaul reveals that, in her own supremely self-interested way, she holds Coriolanus in high regard. He is a fantastic investment, and his time in District 12 has proven that he can thrive even in conditions of deprivation and uncertainty despite his Capitol pedigree. Dr. Gaul's own Capitol elitism is on display in her reference to the 'imbeciles' from the Districts. How much, however, did Coryo really learn from his time in the Districts? His stint as a Peacemaker is construed as a necessary toughening in The Ballad, but the events of the later Hunger Games trilogy suggest that Coriolanus comes to underestimate those District 'imbeciles' at his own peril."]
   ]

In [7]:
max_average = 0

In [8]:
def find_highest(average_score):
    global max_average
    if average_score > max_average:
        max_average = average_score
        print("This is the new best value!")

In [9]:
def dictionary(result):
    dict_result = dict(result)
    average_score = sum(dict_result.values()) / len(dict_result)
    print(f"The average score is: {average_score}")
    find_highest(average_score)
    return average_score, dict_result

### General answer by LLMs

In [10]:
template = """{question}"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
llm_chain =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm}
)
llm_chain_gpt4 =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm_gpt4}
)

In [12]:
answers_llm = []
contexts_llm = [[""],[""],[""],[""],[""],[""],[""],[""],[""]]

In [13]:
for query in questions:
    response = llm_chain.invoke({"question": query})
    answers_llm.append(response["response"].content)

In [16]:
llm_results = evaluation_llm(questions, answers_llm, contexts_llm, ground_truths)
print(llm_results)
avg_llm_results ,dict_llm_results = dictionary(llm_results)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 36/36 [00:07<00:00,  4.62it/s]


{'faithfulness': 0.8114, 'answer_relevancy': 0.8701, 'answer_similarity': 0.8660, 'answer_correctness': 0.6800}
The average score is: 0.8068829769941132
This is the new best value!


In [17]:
answers_llm_gpt4 = []
for query in questions:
    response = llm_chain_gpt4.invoke({"question": query})
    answers_llm_gpt4.append(response["response"].content)
llm_results_gpt4 = evaluation_llm(questions, answers_llm_gpt4, contexts_llm, ground_truths)
print(llm_results_gpt4)
avg_llm_results_gpt4 ,dict_llm_results_gpt4 = dictionary(llm_results_gpt4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 36/36 [00:07<00:00,  4.79it/s]


{'faithfulness': 0.8631, 'answer_relevancy': 0.8825, 'answer_similarity': 0.8583, 'answer_correctness': 0.6600}
The average score is: 0.8159797900896288
This is the new best value!


# RAG

### Naive RAG

In [18]:
def retrieval_chain(prompt, retriever, llm):
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
    )
    return retrieval_augmented_qa_chain

In [19]:
loader = PyPDFLoader(r"..\ballad\the_ballad_of_songbirds_and_snakes.pdf")
documents = loader.load()

In [ ]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = CharacterTextSplitter()
chunks = text_splitter.split_documents(documents)
db_naive = Chroma.from_documents(chunks, embeddings_client, persist_directory = "../ballad/vectordb/naive")
db_naive.persist()
retriever_naive = db_naive.as_retriever()

In [20]:
db_naive = Chroma(persist_directory = "../ballad/vectordb/naive", embedding_function=embeddings_client)
retriever_naive = db_naive.as_retriever()

In [21]:
template = """{question}. 
Provided context {context}."""
prompt = ChatPromptTemplate.from_template(template)

In [22]:
answers_naive = []
contexts_naive = []
for query in questions:
    try:  
        response = retrieval_chain(prompt, retriever_naive, llm).invoke({"question": query})
        # Access the response content
        answers_naive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_naive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_naive.append("No answer")
        context_full = retriever_naive.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_naive.append(context_content)

In [23]:
result_naive_rag = evaluation_rag(questions, answers_naive, contexts_naive, ground_truths)
print(result_naive_rag)
avg_result_naive_rag, dict_result_naive_rag = dictionary(result_naive_rag)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:10<00:00,  5.35it/s]


{'faithfulness': 0.7556, 'answer_relevancy': 0.8829, 'context_precision': 0.6019, 'context_recall': 0.6111, 'answer_similarity': 0.8727, 'answer_correctness': 0.6182}
The average score is: 0.7236995269750812


### recursive splitter

In [ ]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder()
chunks_r = text_splitter.split_documents(documents)
db_basic = Chroma.from_documents(chunks_r, embeddings_client, persist_directory = "../ballad/vectordb/recursive_basic")
db_basic.persist()
retriever_basic = db_basic.as_retriever()

In [24]:
db_basic = Chroma(persist_directory = "../ballad/vectordb/recursive_basic", embedding_function=embeddings_client)
retriever_basic = db_basic.as_retriever()

In [25]:
answers_recursive = []
contexts_recursive = []
for query in questions:
    try:  
        response = retrieval_chain(prompt, retriever_basic, llm).invoke({"question": query})
        # Access the response content
        answers_recursive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_recursive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_recursive.append("No answer")
        context_full = retriever_basic.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_recursive.append(context_content)

In [26]:
result_recursive = evaluation_rag(questions, answers_recursive, contexts_recursive, ground_truths)
print(result_recursive)
avg_result_recursive, dict_result_recursive = dictionary(result_recursive)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

{'faithfulness': 0.7311, 'answer_relevancy': 0.7567, 'context_precision': 0.4875, 'context_recall': 0.6765, 'answer_similarity': 0.7017, 'answer_correctness': 0.7877}
The average score is: 0.6902103845750811


### chunk sizes

In [27]:
def run_and_evaluate(retriever, prompt, llm):
    answers_recursive = []
    contexts_recursive = []

    for query in questions:
        try:  
            response = retrieval_chain(prompt, retriever, llm).invoke({"question": query})
            # Access the response content
            answers_recursive.append(response["response"].content)
            # Access the context content
            context_content = [context.page_content for context in response["context"]]
            contexts_recursive.append(context_content)  
        except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_recursive.append("No answer")
            context_full = retriever.get_relevant_documents(query)
            context_content = [context.page_content for context in context_full]
            contexts_recursive.append(context_content)


    result = evaluation_rag(questions, answers_recursive, contexts_recursive, ground_truths)
    return result

chunk = 1000

In [ ]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap = 100)
chunks_1000 = text_splitter.split_documents(documents)
print(len(chunks_1000))
db_1000 = Chroma.from_documents(chunks_1000, embeddings_client, persist_directory = "../ballad/vectordb/recursive_1000")
db_1000.persist()

In [28]:
db_1000 = Chroma(persist_directory = "../ballad/vectordb/recursive_1000", embedding_function=embeddings_client)

In [29]:
retriever_1000 = db_1000.as_retriever()
result_1000 = run_and_evaluate(retriever_1000, prompt, llm)
print("CHUNK SIZE 1000")
print(result_1000)
avg_result_1000, dict_result_1000 = dictionary(result_1000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

CHUNK SIZE 1000
{'faithfulness': 0.8073, 'answer_relevancy': 0.7830, 'context_precision': 0.4722, 'context_recall': 0.7878, 'answer_similarity': 0.6856, 'answer_correctness': 0.8029}
The average score is: 0.7231345250780993


chunk = 500

In [33]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500, chunk_overlap = 50)
chunks_500 = text_splitter.split_documents(documents)
print(len(chunks_500))
db_500 = Chroma.from_documents(chunks_500, embeddings_client, persist_directory = "../ballad/vectordb/recursive_500")
db_500.persist()

107


In [30]:
db_500 = Chroma(persist_directory = "../ballad/vectordb/recursive_500", embedding_function=embeddings_client)

In [31]:
retriever_500 = db_500.as_retriever()
result_500 = run_and_evaluate(retriever_500, prompt, llm)
print("CHUNK SIZE 500")
print(result_500)
avg_result_500, dict_result_500 = dictionary(result_500)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Task exception was never retrieved
future: <Task finished name='Task-202' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
RuntimeError: Event loop is closed
Task exception was never retrieved
future: <Task finished name='Task-203' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
RuntimeError: Event loop is closed
Task exception was never retrieved
future: <Task finished name='Task-204' coro=<AsyncClient.aclose() done, defined

CHUNK SIZE 500
{'faithfulness': 0.8163, 'answer_relevancy': 0.9034, 'context_precision': 0.4012, 'context_recall': 0.6481, 'answer_similarity': 0.8669, 'answer_correctness': 0.6844}
The average score is: 0.720067425291858


chunk = 2000

In [35]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 2000, chunk_overlap = 200)
chunks_2000 = text_splitter.split_documents(documents)
print(len(chunks_2000))
db_2000 = Chroma.from_documents(chunks_2000, embeddings_client, persist_directory = "../ballad/vectordb/recursive_2000")


37


In [32]:
db_2000 = Chroma(persist_directory = "../ballad/vectordb/recursive_2000", embedding_function=embeddings_client)

In [33]:
retriever_2000 = db_2000.as_retriever()
result_2000 = run_and_evaluate(retriever_2000, prompt, llm)
print("CHUNK SIZE 2000")
print(result_2000)
avg_result_2000, dict_result_2000 = dictionary(result_2000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

CHUNK SIZE 2000
{'faithfulness': 0.8064, 'answer_relevancy': 0.7431, 'context_precision': 0.4389, 'context_recall': 0.6734, 'answer_similarity': 0.6855, 'answer_correctness': 0.7563}
The average score is: 0.6839296969751175


chunk = 3000

In [38]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 3000, chunk_overlap = 300)
chunks_3000 = text_splitter.split_documents(documents)
print(len(chunks_3000))
db_3000 = Chroma.from_documents(chunks_3000, embeddings_client, persist_directory = "../ballad/vectordb/recursive_3000")

32


In [34]:
db_3000 = Chroma(persist_directory = "../ballad/vectordb/recursive_3000", embedding_function=embeddings_client)

In [35]:
retriever_3000 = db_3000.as_retriever()
result_3000 = run_and_evaluate(retriever_3000, prompt, llm)
print("CHUNK SIZE 3000")
print(result_3000)
avg_result_3000 ,dict_result_3000 = dictionary(result_3000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

CHUNK SIZE 3000
{'faithfulness': 0.8087, 'answer_relevancy': 0.7442, 'context_precision': 0.5190, 'context_recall': 0.6554, 'answer_similarity': 0.7109, 'answer_correctness': 0.8236}
The average score is: 0.7103154417058323


### now time to look for different top-k

Note: We continue with the size chunk of 1000 as it had the highest average score

In [36]:
retriever_3 = db_1000.as_retriever(search_kwargs={"k": 3})
result_3 = run_and_evaluate(retriever_3, prompt, llm)
print("CHUNK SIZE 1000, K=3")
print(result_3)
avg_result_3, dict_result_3 = dictionary(result_3)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:09<00:00,  5.72it/s]


CHUNK SIZE 1000, K=3
{'faithfulness': 0.8200, 'answer_relevancy': 0.9060, 'context_precision': 0.5370, 'context_recall': 0.7963, 'answer_similarity': 0.8722, 'answer_correctness': 0.6900}
The average score is: 0.7702549797617143


In [37]:
retriever_5 = db_1000.as_retriever(search_kwargs={"k": 5})
result_5 = run_and_evaluate(retriever_5, prompt, llm)
print("CHUNK SIZE 1000, K=5")
print(result_5)
avg_result_5, dict_result_5 = dictionary(result_5)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:10<00:00,  5.31it/s]


CHUNK SIZE 1000, K=5
{'faithfulness': 0.8667, 'answer_relevancy': 0.8925, 'context_precision': 0.4636, 'context_recall': 0.6204, 'answer_similarity': 0.8748, 'answer_correctness': 0.6645}
The average score is: 0.7303917310800102


In [38]:
retriever_6= db_1000.as_retriever(search_kwargs={"k": 6})
result_6 = run_and_evaluate(retriever_6, prompt, llm)
print("CHUNK SIZE 1000, K=5")
print(result_6)
avg_result_6, dict_result_6 = dictionary(result_6)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:10<00:00,  5.29it/s]


CHUNK SIZE 1000, K=5
{'faithfulness': 0.8667, 'answer_relevancy': 0.9038, 'context_precision': 0.4434, 'context_recall': 0.6481, 'answer_similarity': 0.8671, 'answer_correctness': 0.6306}
The average score is: 0.7266041952543404


In [39]:
retriever_7= db_1000.as_retriever(search_kwargs={"k": 7})
result_7 = run_and_evaluate(retriever_7, prompt, llm)
print("CHUNK SIZE 1000, K=7")
print(result_7)
avg_result_7, dict_result_7 = dictionary(result_7)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

CHUNK SIZE 1000, K=7
{'faithfulness': 0.9253, 'answer_relevancy': 0.7778, 'context_precision': 0.4403, 'context_recall': 0.8073, 'answer_similarity': 0.7465, 'answer_correctness': 0.5476}
The average score is: 0.7074589251052892


In [41]:
retriever_2 = db_1000.as_retriever(search_kwargs={"k": 2})
result_2 = run_and_evaluate(retriever_2, prompt, llm)
print("CHUNK SIZE 1000, K=2")
print(result_2)
avg_result_2, dict_result_2 = dictionary(result_2)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:11<00:00,  4.87it/s]


CHUNK SIZE 1000, K=2
{'faithfulness': 0.6167, 'answer_relevancy': 0.8891, 'context_precision': 0.5000, 'context_recall': 0.7778, 'answer_similarity': 0.8708, 'answer_correctness': 0.6751}
The average score is: 0.7215756009304392


### look for different retrievers

3 chunks was the best score

#### parent document retriever

In [42]:
parent_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=200, chunk_overlap = 0)

# vectorstore = Chroma(collection_name="split_parents",persist_directory = "../ballad/vectordb/parent", embedding_function=embeddings_client)
# vectorstore.persist()
vectorstore = Chroma(persist_directory = "../ballad/vectordb/parent", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={"k": 4}
)
parent_document_retriever.add_documents(documents)
result_parent = run_and_evaluate(parent_document_retriever, prompt, llm)
print(result_parent)
avg_result_parent, dict_result_parent = dictionary(result_parent)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:08<00:00,  6.11it/s]


{'faithfulness': 0.6056, 'answer_relevancy': 0.8858, 'context_precision': 0.5556, 'context_recall': 0.4167, 'answer_similarity': 0.8703, 'answer_correctness': 0.6878}
The average score is: 0.6702675283107856


In [43]:
parent_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap = 50)
child_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap = 0)

# vectorstore = Chroma(collection_name="split_parents_small",persist_directory = "../ballad/vectordb/parent_small", embedding_function=embeddings_client)
# vectorstore.persist()
vectorstore = Chroma(persist_directory = "../ballad/vectordb/parent_small", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever_small = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_small,
    parent_splitter=parent_splitter_small,
    search_kwargs={"k": 4}
)
parent_document_retriever_small.add_documents(documents)
result_parent_small = run_and_evaluate(parent_document_retriever_small, prompt, llm)
print(result_parent_small)
avg_result_parent_small, dict_result_parent_small = dictionary(result_parent_small)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:10<00:00,  4.97it/s]


{'faithfulness': 0.8619, 'answer_relevancy': 0.8815, 'context_precision': 0.3333, 'context_recall': 0.5556, 'answer_similarity': 0.8801, 'answer_correctness': 0.7004}
The average score is: 0.7021394525630296


In [44]:
parent_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=3000)
child_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300, chunk_overlap = 0)

# vectorstore = Chroma(collection_name="split_parents_large",persist_directory = "../ballad/vectordb/parent_large", embedding_function=embeddings_client)
# vectorstore.persist()
vectorstore = Chroma(persist_directory = "../ballad/vectordb/parent_large", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever_large = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_large,
    parent_splitter=parent_splitter_large,
    search_kwargs={"k": 4}
)
parent_document_retriever_large.add_documents(documents)
result_parent_large = run_and_evaluate(parent_document_retriever_large, prompt, llm)
print(result_parent_large)
avg_result_parent_large, dict_result_parent_large = dictionary(result_parent_large)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

{'faithfulness': 0.9333, 'answer_relevancy': 0.9006, 'context_precision': 0.5185, 'context_recall': 0.7500, 'answer_similarity': 0.8543, 'answer_correctness': 0.6110}
The average score is: 0.76130090703488


#### Maximum marginal relevance retrieval

In [45]:
retriever_mmr = db_1000.as_retriever(search_type="mmr",search_kwargs={"k": 3})
result_mmr = run_and_evaluate(retriever_mmr, prompt, llm)
print("Marginal relevance")
print(result_mmr)
average_score_mmr, dict_result_mmr = dictionary(result_mmr)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:11<00:00,  4.87it/s]


Marginal relevance
{'faithfulness': 0.6400, 'answer_relevancy': 0.9034, 'context_precision': 0.5000, 'context_recall': 0.6111, 'answer_similarity': 0.8681, 'answer_correctness': 0.6699}
The average score is: 0.6987434556587875


In [46]:
retriever_mmr_4 = db_1000.as_retriever(search_type="mmr",search_kwargs={"k": 4})
result_mmr_4 = run_and_evaluate(retriever_mmr_4, prompt, llm)
print("Marginal relevance")
print(result_mmr_4)
average_score_mmr_4, dict_result_mmr_4 = dictionary(result_mmr_4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

Marginal relevance
{'faithfulness': 0.7406, 'answer_relevancy': 0.7681, 'context_precision': 0.6815, 'context_recall': 0.6941, 'answer_similarity': 0.7529, 'answer_correctness': 0.7410}
The average score is: 0.7296927753868454


In [49]:
retriever_mmr_5 = db_1000.as_retriever(search_type="mmr",search_kwargs={"k": 5})
result_mmr_5 = run_and_evaluate(retriever_mmr_5, prompt, llm)
print("Marginal relevance")
print(result_mmr_5)
average_score_mmr_5, dict_result_mmr_5 = dictionary(result_mmr_5)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

Marginal relevance
{'faithfulness': 0.7226, 'answer_relevancy': 0.6523, 'context_precision': 0.5508, 'context_recall': 0.5813, 'answer_similarity': 0.9035, 'answer_correctness': 0.6234}
The average score is: 0.6723002779387696


#### BM25

In [47]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap = 100)
chunks_1000 = text_splitter.split_documents(documents)

In [48]:
retriever_bm25 = BM25Retriever.from_documents(chunks_1000, search_kwargs = {"k": 3})
result_bm25 = run_and_evaluate(retriever_bm25, prompt, llm)
print("BM25")
print(result_bm25)
average_score_bm25, dict_result_bm25 = dictionary(result_bm25)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:08<00:00,  6.34it/s]


BM25
{'faithfulness': 0.7074, 'answer_relevancy': 0.8639, 'context_precision': 0.6667, 'context_recall': 0.6667, 'answer_similarity': 0.8815, 'answer_correctness': 0.6775}
The average score is: 0.7439547122210094


#### Ensambler - Hybrid

In [50]:
ensemble_retriever_1 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.75, 0.25])
result_ensemble1 = run_and_evaluate(ensemble_retriever_1, prompt, llm)
print("Ensambler 75/25")
print(result_ensemble1)
average_score_ensambler1, dict_result_ensemble1 = dictionary(result_ensemble1)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:07<00:00,  7.39it/s]


Ensambler 75/25
{'faithfulness': 0.8963, 'answer_relevancy': 0.9131, 'context_precision': 0.4892, 'context_recall': 0.7500, 'answer_similarity': 0.8615, 'answer_correctness': 0.6664}
The average score is: 0.7627580910110087


In [51]:
ensemble_retriever_2 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.5, 0.5])
result_ensemble2 = run_and_evaluate(ensemble_retriever_2, prompt, llm)
print("Ensambler 50/50")
print(result_ensemble2)
avg_result_ensemble2, dict_result_ensemble2 = dictionary(result_ensemble2)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Task exception was never retrieved
future: <Task finished name='Task-1137' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
RuntimeError: Event loop is closed
Task exception was never retrieved
future: <Task finished name='Task-1138' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
RuntimeError: Event loop is closed
Task exception was never retrieved
future: <Task finished name='Task-1139' coro=<AsyncClient.aclose() done, defi

Ensambler 50/50
{'faithfulness': 0.8714, 'answer_relevancy': 0.8963, 'context_precision': 0.4432, 'context_recall': 0.6963, 'answer_similarity': 0.8763, 'answer_correctness': 0.6891}
The average score is: 0.7454412490331638


In [52]:
ensemble_retriever_3 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.25,0.75])
result_ensemble3 = run_and_evaluate(ensemble_retriever_3, prompt, llm)
print("Ensambler 25/75")
print(result_ensemble3)
avg_result_ensemble3, dict_result_ensemble3 = dictionary(result_ensemble3)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:08<00:00,  6.05it/s]


Ensambler 25/75
{'faithfulness': 0.9613, 'answer_relevancy': 0.9009, 'context_precision': 0.4486, 'context_recall': 0.6963, 'answer_similarity': 0.8728, 'answer_correctness': 0.6790}
The average score is: 0.7598177392513458


In [53]:
ensemble_retriever_4 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.1,0.9])
result_ensemble4 = run_and_evaluate(ensemble_retriever_4, prompt, llm)
print("Ensambler 10/90")
print(result_ensemble4)
avg_result_ensemble4, dict_result_ensemble4 = dictionary(result_ensemble4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:09<00:00,  5.88it/s]


Ensambler 10/90
{'faithfulness': 0.9613, 'answer_relevancy': 0.8993, 'context_precision': 0.4486, 'context_recall': 0.7241, 'answer_similarity': 0.8728, 'answer_correctness': 0.6915}
The average score is: 0.766267291905808


#### Multi-stage - reranker

In [54]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [55]:
retriever_context = retriever_3
compressor = CohereRerank(top_n = 3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_context
)

result_compression = run_and_evaluate(compression_retriever, prompt, llm)
print("Reranker")
print(result_compression)
avg_result_compression, dict_result_compression = dictionary(result_compression)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:08<00:00,  6.67it/s]


Reranker
{'faithfulness': 0.8286, 'answer_relevancy': 0.8882, 'context_precision': 0.5185, 'context_recall': 0.7407, 'answer_similarity': 0.8663, 'answer_correctness': 0.6374}
The average score is: 0.7466056282196497


In [56]:
retriever_context_more = retriever_1000
compressor = CohereRerank(top_n = 3)
compression_retriever_more = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_context_more
)

result_compression_more = run_and_evaluate(compression_retriever_more, prompt, llm)
print("Reranker")
print(result_compression_more)
avg_result_compression_more, dict_result_compression_more = dictionary(result_compression_more)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'm

Reranker
{'faithfulness': 0.6468, 'answer_relevancy': 0.8313, 'context_precision': 0.6104, 'context_recall': 0.5648, 'answer_similarity': 0.8625, 'answer_correctness': 0.7789}
The average score is: 0.7157937819391315


#### creating context by remaking the query

In [57]:
template_context = "Generate a search query to fetch the relevant documents using the user's {question}. Craft a query that specifically targets the keywords in the question. In the answer provide only the query."
prompt_context = ChatPromptTemplate.from_template(template_context)

In [58]:
answers_final = []
contexts_final = []
llm_for_context =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt_context | llm}
)
for query in questions:
    response_check = llm_for_context.invoke({"question": query})
    search_query = response_check["response"].content
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("context"), "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
)
    docs = retriever_3.get_relevant_documents(search_query)
    formatted_docs = []
    for doc in docs:
        resulting_doc = doc.page_content
        formatted_docs.append(resulting_doc)
    try:  
            response = retrieval_augmented_qa_chain.invoke({"context": formatted_docs, "question": query})
            # Access the response content
            answers_final.append(response["response"].content)
            contexts_final.append(formatted_docs)  
    except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_final.append("No answer")
            contexts_final.append(formatted_docs)


result_search_query = evaluation_rag(questions, answers_final, contexts_final, ground_truths)
print(result_search_query)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  96%|█████████▋| 52/54 [00:08<00:00,  4.30it/s]Runner in Executor raised an exception
openai.BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'seve

{'faithfulness': 0.6627, 'answer_relevancy': 0.6392, 'context_precision': 0.6711, 'context_recall': 0.4074, 'answer_similarity': 0.5373, 'answer_correctness': 0.8619}


In [59]:
avg_result_search_query, dict_result_search_query = dictionary(result_search_query)

The average score is: 0.6299341753432749


### change model to GPT-4

In [60]:
result_3_gpt4 = run_and_evaluate(retriever_3, prompt, llm_gpt4)
print("k=3 + gpt-4")
print(result_3_gpt4)
avg_result_3_gpt4, dict_result_3_gpt4 = dictionary(result_3_gpt4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:08<00:00,  6.40it/s]


k=3 + gpt-4
{'faithfulness': 0.8267, 'answer_relevancy': 0.8664, 'context_precision': 0.4815, 'context_recall': 0.7685, 'answer_similarity': 0.8805, 'answer_correctness': 0.6948}
The average score is: 0.753078100078859


In [61]:
template = """ Answer the following usery query {question}. 
Answer should use the provided context and general knowledge. 
Context {context}."""
prompt_better = ChatPromptTemplate.from_template(template)

In [62]:
result_3_gpt4_prompt = run_and_evaluate(retriever_3, prompt_better, llm_gpt4)
print("k=3 + gpt-4")
print(result_3_gpt4_prompt)
avg_result_3_gpt4_prompt, dict_result_3_gpt4_prompt = dictionary(result_3_gpt4_prompt)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 54/54 [00:10<00:00,  5.00it/s]


k=3 + gpt-4
{'faithfulness': 0.6593, 'answer_relevancy': 0.7789, 'context_precision': 0.5926, 'context_recall': 0.8056, 'answer_similarity': 0.8779, 'answer_correctness': 0.6865}
The average score is: 0.7334556911163995
